## NBG Gravestone Image Pipeline

Goal: Set up a pipeline to Claude to identify non-text parts of the image (shape, icongraphy, etc) as well as OCR to extract key text and information such as name, birth and death dates, and age. 

Currently, the model costs about 1.7 cents and takes around 20 seconds per image. Both of these could be reduced in the future (combining the prompts would use slightly less tokens and many requests, but may require a trade off in accuracy). Switching LLM's could also result in this. 

To use the pipeline, make sure you are following instructions in the README and here - once you've set up your environment and API Key, you should be able to just run these cells with your images. 

In [2]:
# Having errors? Want to see the code? Look at llm_helper functions! 
from llm_helper_functions import *
from ocr_helper_functions import * 

#TODO
#
# Run / Test / Find Errors
    # Edit the transcription prompt to add ? for unknown characters? 
    # Needs Error checking to save data even if something goes wrong 

# Make the API Set up thing better / include constants for those paths as well 

### Folder and API Set Up

In [3]:
# Note there is a 5MB limit on images
INPUT_FOLDER = "../data/input/" # TODO change to ..data/input/
OUTPUT_FOLDER = "../data/output/"
OUTPUT_FILENAME = "results.csv"

API_KEY = get_api_key("credentials.txt") 
HEADERS = {
    "Content-Type": "application/json",
    "x-api-key": API_KEY,
    "anthropic-version": "2023-06-01"
}
MODEL = "" #TODO Allow to more easily change out models, for now you have to go to llm_helper_functions to change it 

### Prompts:
Feel free to change or add more!

In [4]:
# All of these prompts will be accompanied by the image
ICON_PROMPT = "Hi! Can you identify the iconography of this gravestone? Most of the icongraphy should be towards the top of the stone. " \
"If there is no icongoraphy, just say None. Please only return exactly what the iconography is. Do not say anything else in your answer."

SHAPE_PROMPT = "Hi! Can you identify the shape of this gravestone? Common shapes are Square Top, Check Top, Ogee, Arc Top, Arc Top with Shoulders, " \
"Half-Round, Half Ogee, Arc Top with Scotia Shoulders, and Peon Top Please only return exactly what the shape is. Do not say anything else in your answer."

MATERIAL_PROMPT = "Hi! Can you tell me which material this gravestone is made of? It should be one of granite, marble, or slate. " \
"Please only return exactly what the material is. Do not say anything else in your answer." 

TRANSCRIPTION_PROMPT = "Hi! Can you transcribe the text on this gravestone? Please deliminate each line of the transcription with a hyphen. " \
"Please only return the transcription. Do not say anything else in your answer."

YOUR_PROMPT_HERE = ""

# You can add your prompt variable and corresponding column here
PROMPTS = [ICON_PROMPT, SHAPE_PROMPT, MATERIAL_PROMPT, TRANSCRIPTION_PROMPT] # Dont put the info prompt in here
COLUMNS = ["Image Name", "Iconography Description", "Shape Description", "Material", "Claude Transcription"] # Don't change first/last column order

# Separate Task to translate the transcription
INFO_PROMPT = "Hi! The following is a transcription from a gravestone. Each line is separated by a newline character." \
"Can you tell me the first name, middle name, last name, date of birth, date of death, age at death." \
"The information will not be labeled. You might have to calculate age on death, birth year, or death year based on the other two. If there is information missing for a field, put None. Please only return exactly " \
"the information requested, in order separated by a comma. Only do this for the first person if there are multiple. Do not say anything else in your answer. Here is the Transcription: "

# You can add info to extract here if you change the prompt above 
INFO_COLUMNS = ["First Name", "Middle Name", "Last Name", "Date of Birth", "Date of Death", "Age at Death", "Claude Transcription"]


### Claude Pipeline

In [ ]:
df_desc = gravestone_desc(INPUT_FOLDER, PROMPTS, COLUMNS, HEADERS, debug=False)
df_desc.to_csv(OUTPUT_FOLDER + OUTPUT_FILENAME)
df_desc.head()

hi
hi2
hi


KeyboardInterrupt: 

In [5]:
df_desc = pd.read_csv(OUTPUT_FOLDER + "results.csv")
df_desc.head()

,Unnamed: 0,Image Name,Iconography Description,Shape Description,Material,Claude Transcription
0,0,_DSC0437.jpeg,NaN,Square Top,Marble,ERECTED\n- to the Memory of\n- [unclear text]\...
1,1,_DSC0421.jpeg,NaN,Square Top,Slate,"In Memory\n-of\n-SARAH THURBER\n-BENSON,\n-rel..."
2,2,.DS_Store,I don't see any image attached to your message...,I don't see an image of a gravestone in your m...,I don't see an image of a gravestone in your m...,I don't see any image attached to your message...
3,3,_DSC0420.jpeg,NaN,Square Top,Slate,In Memory\nof\nFRANCES BENSON\neldest daughter...
4,4,_DSC0416.jpeg,NaN,Square Top,Slate,"In Memory\n-of\n-HENRY E. BENSON,\n-youngest s..."


### Get Data from Transcription

In [6]:
df_info = transcription_info(df_desc["Claude Transcription"], INFO_PROMPT, INFO_COLUMNS, HEADERS, debug=False)
df_all = pd.concat([df_desc, df_info])
df_all.to_csv(OUTPUT_FOLDER + OUTPUT_FILENAME)

Hi! The following is a transcription from a gravestone. Each line is separated by a newline character.Can you tell me the first name, middle name, last name, date of birth, date of death, age at death.The information will not be labeled. You might have to calculate age on death, birth year, or death year based on the other two. If there is information missing for a field, put None. Please only return exactly the information requested, in order separated by a comma. Only do this for the first person if there are multiple. Do not say anything else in your answer. Here is the Transcription: ERECTED
- to the Memory of
- [unclear text]
- In the 46th year
- WILLIAM McDONALD
- & daughter of
- JAMES BERKSHIRE
- She died
- Aug 30-1856
- in the 33d year
- of her age
Hi! The following is a transcription from a gravestone. Each line is separated by a newline character.Can you tell me the first name, middle name, last name, date of birth, date of death, age at death.The information will not be labe

ValueError: 7 columns passed, passed data had 9 columns

## OCR

In [ ]:
# Old OCR model - not good, but you can run it if curious 
# df = tesseract_ocr(INPUT_FOLDER)
df = process_easy_ocr(INPUT_FOLDER)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 MB 37.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 37.7 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.6 MB/s eta 0:00:00
Processing _DSC0470.jpeg...
Processing _DSC0469.jpeg...


In [ ]:
df.to_csv(OUTPUT_FOLDER + "ocr_results.csv")
df.head()

Processing _DSC0421.jpeg...

🔎 OCR Output:


Error: command buffer exited with error status.
	The Metal Performance Shaders operations encoded on it may not have completed.
	Error: 
	(null)
	Internal Error (e00002bd:Internal Error)
	<GFX10_MtlCmdBuffer: 0x7f90827b1e00>
    label = <none> 
    device = <GFX10_MtlDevice: 0x7f90759d0000>
        name = AMD Radeon Pro 5500M 
    commandQueue = <GFXAAMD_MtlCmdQueue: 0x7f91973c0d00>
        label = <none> 
        device = <GFX10_MtlDevice: 0x7f90759d0000>
            name = AMD Radeon Pro 5500M 
    retainedReferences = 1
